In [162]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import math
import time
import datetime

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

In [163]:
# This is needed to display the images.
%matplotlib inline

In [164]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [165]:
# What model to use
MODEL_NAME = 'litter_inference_graph'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('kab_training', 'litter_detection_map.pbtxt')

NUM_CLASSES = 1

config = tf.ConfigProto()
config.gpu_options.force_gpu_compatible = True
config.gpu_options.per_process_gpu_memory_fraction = 1

In [166]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [167]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [168]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def rescale(im_width, im_height, image):
    aspectRatio = im_width / im_height
    new_width = ( 640 * aspectRatio )
    new_height = ( new_width / aspectRatio )
    image = image.resize((int(new_width), int(new_height)), resample=0)
    return image

In [169]:
# The images to be tested are located in this directory
PATH_TO_TEST_IMAGES_DIR = 'SanMarcos'

TEST_IMAGE_PATHS = []
for root, dirs, filenames in os.walk(PATH_TO_TEST_IMAGES_DIR):
    for f in filenames:
        file_path = os.path.join(PATH_TO_TEST_IMAGES_DIR, f)
        TEST_IMAGE_PATHS.append(file_path)

# Size, in inches, of the output images.
IMAGE_SIZE = (16, 12)

# THRESHOLD
THRESHOLD = 0.65 # The minimum score threshold for showing detections. default = 0.5
MAX_BOXES = 30  # The maximum number of boxes to draw for detections. default = 30

In [170]:
def run_inference_for_single_image(image, graph, sess):
  # Get handles to input and output tensors
  ops = tf.get_default_graph().get_operations()
  all_tensor_names = {output.name for op in ops for output in op.outputs}
  tensor_dict = {}
    
  for key in [ 'num_detections', 'detection_boxes', 'detection_scores', 'detection_classes', 'detection_masks']:
    tensor_name = key + ':0'
    if tensor_name in all_tensor_names:
      tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
    
  if 'detection_masks' in tensor_dict:
    # The following processing is only for single image
    detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
    detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
    # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
    real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
    detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
    detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks( detection_masks, detection_boxes, image.shape[0], image.shape[1])
    detection_masks_reframed = tf.cast(tf.greater(detection_masks_reframed, 0.5), tf.uint8)
    # Follow the convention by adding back the batch dimension
    tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
    
  image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

  # Run inference
  output_dict = sess.run(tensor_dict,feed_dict={image_tensor: np.expand_dims(image, 0)})

  # all outputs are float32 numpy arrays, so convert types as appropriate
  output_dict['num_detections'] = int(output_dict['num_detections'][0])
  output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
  output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
  output_dict['detection_scores'] = output_dict['detection_scores'][0]
    
  if 'detection_masks' in output_dict:
    output_dict['detection_masks'] = output_dict['detection_masks'][0]
    
  return output_dict

In [171]:
print('\n// Running object detection algorithm')

with detection_graph.as_default():
    sess = tf.Session(graph=detection_graph,config=config)
    with tf.device('/device:GPU:0'):
        for i, image_path in enumerate(TEST_IMAGE_PATHS):
            start = time.time()

            image = Image.open(image_path)
            im_width, im_height = image.size

            # rescale image if bigger than 640 x 640
            if (im_width > 640 or im_height > 640):
                image = rescale(im_width, im_height, image)

            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            
            # Actual detection.
            output_dict = run_inference_for_single_image(image_np, detection_graph, sess)
            total = time.time() - start
    
            # Visualization of the results of a detection.
            #vis_util.visualize_boxes_and_labels_on_image_array(
            #  image_np,
            #  output_dict['detection_boxes'],
            #  output_dict['detection_classes'],
            #  output_dict['detection_scores'],
            #  category_index,
            #  instance_masks=output_dict.get('detection_masks'),
            #  use_normalized_coordinates=True,
            #  max_boxes_to_draw=MAX_BOXES,
            #  min_score_thresh=THRESHOLD,
            #  line_thickness=2)

            # Get the number of detections shown on image          
            count = len([i for i in output_dict['detection_scores'] if i >= THRESHOLD])

            # Determine the ranking
            rank_dict = {1: 'Low', 2:'Medium', 3:'High', 4:'Very High'}

            if count in range(0, 3):          # 0 - 2 objects [low]
              rank = 1
            elif count in range(3, 6):        # 3 - 5 objects [medium]
              rank = 2
            elif count in range(6, 9):        # 6 - 8 objects [high]
              rank = 3
            elif count >= 9:                  # 9 + objects [very high]
              rank = 4
            
            d = datetime.datetime.today()
            fd = d.strftime('%y-%m-%d')
            rawCoords = str(image_path)
            rawCoords = rawCoords.split("/")
            
            #take image name, split it from path, truncate file type
            coords = rawCoords[1]
            coords = coords[0:-7]
            coords = coords.split(",")
            #output coords, num litter detected, KAB Rank, todays date
            theRow = str(coords[0])+","+str(coords[1])+ "," + str(rank)+ "," + str(count)+ "," + fd + "\n"
            
            with open("SanMarcosData.txt", "a") as f:
                f.write(theRow)
            # display the image
            # plt.figure(figsize=IMAGE_SIZE)
            # plt.title("Detected: " + theRow, fontsize=15)
            # plt.imshow(image_np)
            
            print("[" + str(i) + "] Processed " + str(image_path) + " \t time = " + str(total))


// Running object detection algorithm
[0] Processed SanMarcos/33.138404,-117.133426_A2.jpg 	 time = 9.652140140533447
[1] Processed SanMarcos/33.128989,-117.139082_A1.jpg 	 time = 6.115059852600098
[2] Processed SanMarcos/33.107626,-117.171682_A1.jpg 	 time = 6.12100076675415
[3] Processed SanMarcos/33.13939,-117.143225_A1.jpg 	 time = 6.153546094894409
[4] Processed SanMarcos/33.138861,-117.18027_A1.jpg 	 time = 6.180891275405884
[5] Processed SanMarcos/33.120714,-117.166082_A1.jpg 	 time = 6.1874072551727295
[6] Processed SanMarcos/33.142516,-117.188855_A1.jpg 	 time = 6.223322868347168
[7] Processed SanMarcos/33.096589,-117.196622_A1.jpg 	 time = 6.2157673835754395
[8] Processed SanMarcos/33.133511,-117.1875_A2.jpg 	 time = 6.221184968948364
[9] Processed SanMarcos/33.121095,-117.166206_A1.jpg 	 time = 6.223287105560303
[10] Processed SanMarcos/33.142093,-117.125183_A1.jpg 	 time = 6.221429109573364
[11] Processed SanMarcos/33.168423,-117.152649_A1.jpg 	 time = 6.237748384475708
[1

[102] Processed SanMarcos/33.132691,-117.126158_A2.jpg 	 time = 6.179736614227295
[103] Processed SanMarcos/33.102434,-117.22292_A2.jpg 	 time = 6.15643048286438
[104] Processed SanMarcos/33.099106,-117.213182_A2.jpg 	 time = 6.170074224472046
[105] Processed SanMarcos/33.138683,-117.217003_A2.jpg 	 time = 6.171313762664795
[106] Processed SanMarcos/33.120747,-117.221336_A1.jpg 	 time = 6.172116041183472
[107] Processed SanMarcos/33.151904,-117.134465_A2.jpg 	 time = 6.147383213043213
[108] Processed SanMarcos/33.128536,-117.184753_A1.jpg 	 time = 6.169148683547974
[109] Processed SanMarcos/33.122086,-117.165751_A2.jpg 	 time = 6.167299509048462
[110] Processed SanMarcos/33.111947,-117.225729_A1.jpg 	 time = 6.166855096817017
[111] Processed SanMarcos/33.094203,-117.20544_A1.jpg 	 time = 6.1665918827056885
[112] Processed SanMarcos/33.123661,-117.19043_A2.jpg 	 time = 6.139590501785278
[113] Processed SanMarcos/33.147462,-117.186377_A1.jpg 	 time = 6.155186891555786
[114] Processed San

[202] Processed SanMarcos/33.110901,-117.225327_A2.jpg 	 time = 6.165365695953369
[203] Processed SanMarcos/33.125992,-117.174263_A2.jpg 	 time = 6.172505617141724
[204] Processed SanMarcos/33.142289,-117.140873_A1.jpg 	 time = 6.182840824127197
[205] Processed SanMarcos/33.143714,-117.203329_A1.jpg 	 time = 6.151564836502075
[206] Processed SanMarcos/33.1078,-117.165079_A1.jpg 	 time = 6.188377857208252
[207] Processed SanMarcos/33.09544,-117.195708_A1.jpg 	 time = 6.156343460083008
[208] Processed SanMarcos/33.131407,-117.204584_A1.jpg 	 time = 6.154420375823975
[209] Processed SanMarcos/33.126919,-117.187759_A2.jpg 	 time = 6.1732261180877686
[210] Processed SanMarcos/33.148614,-117.188893_A2.jpg 	 time = 6.20414662361145
[211] Processed SanMarcos/33.157225,-117.2063_A2.jpg 	 time = 6.185943365097046
[212] Processed SanMarcos/33.156643,-117.132446_A1.jpg 	 time = 6.163746356964111
[213] Processed SanMarcos/33.156776,-117.145107_A2.jpg 	 time = 6.152784109115601
[214] Processed SanMa

[303] Processed SanMarcos/33.140498,-117.136198_A2.jpg 	 time = 6.20639705657959
[304] Processed SanMarcos/33.142304,-117.189658_A2.jpg 	 time = 6.19381308555603
[305] Processed SanMarcos/33.143218,-117.189999_A1.jpg 	 time = 6.183319807052612
[306] Processed SanMarcos/33.134711,-117.127943_A2.jpg 	 time = 6.177660226821899
[307] Processed SanMarcos/33.13921,-117.216133_A2.jpg 	 time = 6.169515132904053
[308] Processed SanMarcos/33.107421,-117.170109_A1.jpg 	 time = 6.187186241149902
[309] Processed SanMarcos/33.153039,-117.144933_A2.jpg 	 time = 6.205570220947266
[310] Processed SanMarcos/33.134739,-117.197823_A2.jpg 	 time = 6.180689334869385
[311] Processed SanMarcos/33.135887,-117.200348_A2.jpg 	 time = 6.177778244018555
[312] Processed SanMarcos/33.10217,-117.180853_A1.jpg 	 time = 6.223199129104614
[313] Processed SanMarcos/33.142289,-117.140873_A2.jpg 	 time = 6.204648971557617
[314] Processed SanMarcos/33.138736,-117.134451_A2.jpg 	 time = 6.197482585906982
[315] Processed SanM

[404] Processed SanMarcos/33.141621,-117.185807_A1.jpg 	 time = 6.179827690124512
[405] Processed SanMarcos/33.121647,-117.169864_A2.jpg 	 time = 6.182952880859375
[406] Processed SanMarcos/33.136594,-117.171721_A2.jpg 	 time = 6.164424180984497
[407] Processed SanMarcos/33.142811,-117.189705_A1.jpg 	 time = 6.194998264312744
[408] Processed SanMarcos/33.133767,-117.125058_A2.jpg 	 time = 6.168747901916504
[409] Processed SanMarcos/33.142735,-117.155636_A2.jpg 	 time = 6.176480054855347
[410] Processed SanMarcos/33.099524,-117.199544_A1.jpg 	 time = 6.169324636459351
[411] Processed SanMarcos/33.142015,-117.159386_A2.jpg 	 time = 6.1791346073150635
[412] Processed SanMarcos/33.164233,-117.188816_A1.jpg 	 time = 6.181682348251343
[413] Processed SanMarcos/33.145459,-117.133144_A1.jpg 	 time = 6.170811414718628
[414] Processed SanMarcos/33.127251,-117.186028_A2.jpg 	 time = 6.209639549255371
[415] Processed SanMarcos/33.149862,-117.188672_A1.jpg 	 time = 6.197854042053223
[416] Processed

...

[3788] Processed SanMarcos/33.132477,-117.196967_A2.jpg 	 time = 6.153425693511963
[3789] Processed SanMarcos/33.154431,-117.141064_A2.jpg 	 time = 6.1664979457855225
[3790] Processed SanMarcos/33.121123,-117.166026_A1.jpg 	 time = 6.177563428878784
[3791] Processed SanMarcos/33.130879,-117.219456_A1.jpg 	 time = 6.171391487121582
[3792] Processed SanMarcos/33.125244,-117.175987_A1.jpg 	 time = 6.174598455429077
[3793] Processed SanMarcos/33.138195,-117.158796_A2.jpg 	 time = 6.181183099746704
[3794] Processed SanMarcos/33.143764,-117.203217_A1.jpg 	 time = 6.150130748748779
[3795] Processed SanMarcos/33.139368,-117.179748_A2.jpg 	 time = 6.177811145782471
[3796] Processed SanMarcos/33.131815,-117.158678_A2.jpg 	 time = 6.170390605926514
[3797] Processed SanMarcos/33.126415,-117.188362_A1.jpg 	 time = 6.166793346405029
[3798] Processed SanMarcos/33.102401,-117.180137_A1.jpg 	 time = 6.226778030395508
[3799] Processed SanMarcos/33.139072,-117.143478_A1.jpg 	 time = 6.201673984527588
[38

[3887] Processed SanMarcos/33.141325,-117.160868_A2.jpg 	 time = 6.179059267044067
[3888] Processed SanMarcos/33.122232,-117.171206_A2.jpg 	 time = 6.158352375030518
[3889] Processed SanMarcos/33.160027,-117.154984_A1.jpg 	 time = 6.148545742034912
[3890] Processed SanMarcos/33.140983,-117.184312_A1.jpg 	 time = 6.18191933631897
[3891] Processed SanMarcos/33.12892,-117.138379_A2.jpg 	 time = 6.175012826919556
[3892] Processed SanMarcos/33.122655,-117.162669_A1.jpg 	 time = 6.162212133407593
[3893] Processed SanMarcos/33.122589,-117.17923_A1.jpg 	 time = 6.15630030632019
[3894] Processed SanMarcos/33.136177,-117.132462_A1.jpg 	 time = 6.1706624031066895
[3895] Processed SanMarcos/33.127918,-117.188538_A2.jpg 	 time = 6.160022497177124
[3896] Processed SanMarcos/33.123981,-117.175636_A2.jpg 	 time = 6.196770906448364
[3897] Processed SanMarcos/33.12298,-117.162508_A1.jpg 	 time = 6.1698808670043945
[3898] Processed SanMarcos/33.135396,-117.129613_A2.jpg 	 time = 6.166631460189819
[3899] 

[3986] Processed SanMarcos/33.122326,-117.162445_A1.jpg 	 time = 6.158395528793335
[3987] Processed SanMarcos/33.102361,-117.186353_A2.jpg 	 time = 6.151167154312134
[3988] Processed SanMarcos/33.142114,-117.198894_A1.jpg 	 time = 6.144322633743286
[3989] Processed SanMarcos/33.142246,-117.160708_A1.jpg 	 time = 6.147889614105225
[3990] Processed SanMarcos/33.150839,-117.156721_A2.jpg 	 time = 6.161006450653076
[3991] Processed SanMarcos/33.120326,-117.165711_A2.jpg 	 time = 6.168283700942993
[3992] Processed SanMarcos/33.149784,-117.139557_A1.jpg 	 time = 6.17128849029541
[3993] Processed SanMarcos/33.134632,-117.153816_A2.jpg 	 time = 6.157519578933716
[3994] Processed SanMarcos/33.121441,-117.192307_A2.jpg 	 time = 6.193400144577026
[3995] Processed SanMarcos/33.13588,-117.150887_A2.jpg 	 time = 6.148073673248291
[3996] Processed SanMarcos/33.127407,-117.19043_A1.jpg 	 time = 6.183568477630615
[3997] Processed SanMarcos/33.146366,-117.12224_A1.jpg 	 time = 6.139227628707886
[3998] P

[4085] Processed SanMarcos/33.103845,-117.178037_A2.jpg 	 time = 6.154844522476196
[4086] Processed SanMarcos/33.155891,-117.198262_A1.jpg 	 time = 6.152533531188965
[4087] Processed SanMarcos/33.117363,-117.177894_A2.jpg 	 time = 6.165566921234131
[4088] Processed SanMarcos/33.160748,-117.154778_A1.jpg 	 time = 6.164026975631714
[4089] Processed SanMarcos/33.155037,-117.189678_A2.jpg 	 time = 6.189659833908081
[4090] Processed SanMarcos/33.115499,-117.226464_A1.jpg 	 time = 6.152040719985962
[4091] Processed SanMarcos/33.140301,-117.142769_A2.jpg 	 time = 6.1397740840911865
[4092] Processed SanMarcos/33.156841,-117.158524_A1.jpg 	 time = 6.183852195739746
[4093] Processed SanMarcos/33.14915,-117.190268_A1.jpg 	 time = 6.171849012374878
[4094] Processed SanMarcos/33.131937,-117.209804_A2.jpg 	 time = 6.183371305465698
[4095] Processed SanMarcos/33.16917,-117.153507_A2.jpg 	 time = 6.193159103393555
[4096] Processed SanMarcos/33.142974,-117.191707_A2.jpg 	 time = 6.149940252304077
[4097

[4184] Processed SanMarcos/33.121123,-117.166026_A2.jpg 	 time = 6.155799388885498
[4185] Processed SanMarcos/33.134186,-117.17793_A2.jpg 	 time = 6.182121753692627
[4186] Processed SanMarcos/33.123524,-117.192551_A1.jpg 	 time = 6.186676502227783
[4187] Processed SanMarcos/33.146982,-117.13675_A1.jpg 	 time = 6.179995536804199
[4188] Processed SanMarcos/33.130181,-117.179743_A1.jpg 	 time = 6.207720518112183
[4189] Processed SanMarcos/33.136875,-117.148956_A1.jpg 	 time = 6.209001541137695
[4190] Processed SanMarcos/33.149875,-117.151159_A2.jpg 	 time = 6.193310499191284
[4191] Processed SanMarcos/33.132715,-117.216464_A2.jpg 	 time = 6.213541507720947
[4192] Processed SanMarcos/33.150799,-117.192291_A2.jpg 	 time = 6.193491697311401
[4193] Processed SanMarcos/33.12515,-117.178504_A2.jpg 	 time = 6.203273296356201
[4194] Processed SanMarcos/33.118503,-117.146726_A1.jpg 	 time = 6.218400716781616
[4195] Processed SanMarcos/33.129763,-117.22868_A1.jpg 	 time = 6.206116199493408
[4196] P